In [38]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline

In [39]:
#откроем данные
data = pd.read_csv('data/unconv.csv')
data.head()

,Well,Por,Perm,AI,Brittle,TOC,VR,Prod
0,1,12.08,2.92,2.80,81.40,1.16,2.31,4165.196191
1,2,12.38,3.53,3.22,46.17,0.89,1.88,3561.146205
2,3,14.02,2.59,4.01,72.80,0.89,2.72,4284.348574
3,4,17.67,6.75,2.63,39.81,1.08,1.88,5098.680869
4,5,17.52,4.57,3.18,10.94,1.51,1.90,3406.132832


## Задание 2.1

In [40]:
#Постройте корреляционную матрицу факторов, включив в неё целевой признак.
C = data.corr()
C

,Well,Por,Perm,AI,Brittle,TOC,VR,Prod
Well,1.000000,0.068927,0.077928,0.041483,-0.079252,0.022624,-0.007279,0.026817
Por,0.068927,1.000000,0.760546,-0.461549,-0.218570,0.711831,0.111860,0.861910
Perm,0.077928,0.760546,1.000000,-0.239636,-0.124017,0.471746,0.051023,0.727426
AI,0.041483,-0.461549,-0.239636,1.000000,0.127599,-0.531864,0.499143,-0.390835
Brittle,-0.079252,-0.218570,-0.124017,0.127599,1.000000,-0.214282,0.317929,0.237155
TOC,0.022624,0.711831,0.471746,-0.531864,-0.214282,1.000000,0.299483,0.654445
VR,-0.007279,0.111860,0.051023,0.499143,0.317929,0.299483,1.000000,0.323182
Prod,0.026817,0.861910,0.727426,-0.390835,0.237155,0.654445,0.323182,1.000000


In [41]:
#Вычислите ранг полученной матрицы корреляций
print('Ранг корреляционной матрицы:', np.linalg.matrix_rank(C))

Ранг корреляционной матрицы: 8


In [42]:
#Вычислите определитель матрицы корреляций
print('Определитель матрицы корреляций:', np.linalg.det(C))

#Определитель матрицы корреляций близок к 0. Это значит, что есть мультиколлинеарные факторы.

Определитель матрицы корреляций: 0.0007299388072652095


## Задание 2.2

In [43]:
#Создайте матрицу наблюдений. Обозначьте её за X, а вектор правильных ответов — за y .

#Добавим вектор единиц и удалим колонку с целевым признаком.
X = np.column_stack((np.ones(data.shape[0]), data.drop(['Prod'], axis=1)))
y = data['Prod']

In [44]:
#Постройте модель линейной регрессии по методу наименьших квадратов. 
#Для этого используйте матричную формулу NumPy. 
#В качестве ответа укажите полученные оценки коэффициентов модели

w_hat = np.linalg.inv(X.T@X)@X.T@y
print(w_hat)

[-1.23230803e+03  5.07003631e-02  2.30179140e+02  1.16239006e+02
 -3.65202301e+02  2.49943700e+01 -7.84009294e+01  7.85259815e+02]


## Задание 2.3

In [45]:
#Постройте прогноз выработки газа для скважины с параметрами, указанными ниже.
#Чему равна абсолютная ошибка построенного вами прогноза для предложенной скважины ?(в миллионах кубических футов в день).
filter = data['Well'] == 106

i = data[filter].index[0]

y_pred = X[i] @ w_hat
print('Ошибка прогноза выработки газа:', abs(y_pred - y[i]))

Ошибка прогноза выработки газа: 25.250970292819147


Метрика MAPE:

![](https://miro.medium.com/max/596/1*vmIM1jePXsDxPpgWEQ35Lg.png)

In [46]:
#Постройте прогноз выработки газа для всех скважин из обучающего набора данных. 
#Чему равно значение метрики MAPE вашей модели? 
sum = 0
n = X.shape[0]
for i in range(0, n):
    y_pred = X[i] @ w_hat
    sum += abs(y_pred - y[i]) / y[i]
    
print('MAPE для модели составляет (%):', sum / n * 100)

MAPE для модели составляет (%): 3.627946845133971


## Задание 2.5

In [47]:
#Исключите из данных сильно коррелированные между собой факторы.

#Сильно коррелируют 3 фактора - Perm, TOC и Por. Удалим первые 2, т.к. Por лучше всего коррелирует с целевым признаком.
#Также удалим идентификатор колонки Well, т.к. от него не зависит целевой признак.
data.drop(['Well', 'Perm', 'TOC'], axis=1, inplace=True)
data.head()

,Por,AI,Brittle,VR,Prod
0,12.08,2.80,81.40,2.31,4165.196191
1,12.38,3.22,46.17,1.88,3561.146205
2,14.02,4.01,72.80,2.72,4284.348574
3,17.67,2.63,39.81,1.88,5098.680869
4,17.52,3.18,10.94,1.90,3406.132832


In [48]:
#Построим матрицу корреляций с целевым признаком.
C = data.corr()
C

,Por,AI,Brittle,VR,Prod
Por,1.000000,-0.461549,-0.218570,0.111860,0.861910
AI,-0.461549,1.000000,0.127599,0.499143,-0.390835
Brittle,-0.218570,0.127599,1.000000,0.317929,0.237155
VR,0.111860,0.499143,0.317929,1.000000,0.323182
Prod,0.861910,-0.390835,0.237155,0.323182,1.000000


In [49]:
#Постройте линейную регрессию на обновлённых после удаления факторов данных по методу наименьших квадратов.

#Добавим вектор единиц и удалим колонку с целевым признаком.
X = np.column_stack((np.ones(data.shape[0]), data.drop(['Prod'], axis=1)))
y = data['Prod']

w_hat = np.linalg.inv(X.T@X)@X.T@y
print(w_hat)

[-1835.44646069   293.03624565  -200.03091206    27.64098209
   517.40272597]


In [50]:
# Сделайте прогноз для всего обучающего набора данных и рассчитайте метрику MAPE (Mean Absolute Percentage Error).
sum = 0
n = X.shape[0]
for i in range(0, n):
    y_pred = X[i] @ w_hat
    sum += abs(y_pred - y[i]) / y[i]
    
print('MAPE для модели составляет (%):', sum / n * 100)

MAPE для модели составляет (%): 4.04413842043617


## Задание 3.1

In [55]:
#Сгенерируйте полиномиальные признаки третьего порядка на факторах, которые вы выбрали для обучения моделей
from sklearn.preprocessing import PolynomialFeatures

X = data.drop(['Prod'], axis=1)

poly = PolynomialFeatures(degree=3, include_bias=False)
X = poly.fit_transform(X)
display(pd.DataFrame(X).head())

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,12.08,2.80,81.40,2.31,145.9264,33.8240,983.3120,27.9048,7.8400,227.9200,...,21.952000,638.176000,18.110400,18552.688000,526.495200,14.941080,539353.144000,15305.967600,434.358540,12.326391
1,12.38,3.22,46.17,1.88,153.2644,39.8636,571.5846,23.2744,10.3684,148.6674,...,33.386248,478.709028,19.492592,6863.973858,279.494712,11.380768,98419.153113,4007.537532,163.183248,6.644672
2,14.02,4.01,72.80,2.72,196.5604,56.2202,1020.6560,38.1344,16.0801,291.9280,...,64.481201,1170.631280,43.737872,21252.358400,794.044160,29.667584,385828.352000,14415.564800,538.603520,20.123648
3,17.67,2.63,39.81,1.88,312.2289,46.4721,703.4427,33.2196,6.9169,104.7003,...,18.191447,275.361789,13.003772,4168.118943,196.836564,9.295472,63092.325141,2979.491868,140.704464,6.644672
4,17.52,3.18,10.94,1.90,306.9504,55.7136,191.6688,33.2880,10.1124,34.7892,...,32.157432,110.629656,19.213560,380.593848,66.099480,11.479800,1309.338584,227.398840,39.493400,6.859000


In [56]:
#Обучите модель линейной регрессии из библиотеки sklearn (LinearRegression) на полученных полиномиальных факторах.
from sklearn.linear_model import LinearRegression

# создаём модель линейной регрессии
lr = LinearRegression()
lr.fit(X, y)
y_pred = lr.predict(X)
print(y_pred[:10], '...')

[4004.29030203 3446.81589884 4408.00806936 5012.67544702 3150.04970867
 4287.92472055 4141.33998162 3435.21890988 4155.00204336 4288.70296274] ...


In [57]:
#Оцените среднее значение метрики MAPE, используя кросс-валидацию на пяти фолдах.
from sklearn.model_selection import cross_validate

lr = LinearRegression()
# оцениваем качество модели на кросс-валидации, метрика — MAPE
cv_results = cross_validate(lr, X, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results['test_score'].mean() * 100))

MAPE на тренировочных фолдах: 1.77 %
MAPE на валидационных фолдах: 2.68 %


## Задание 3.2

In [63]:
# Обучите модель Lasso из библиотеки sklearn на полученных полиномиальных факторах, предварительно стандартизировав факторы с помощью StandardScaler. Коэффициент регуляризации выставите равным 5.
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

X = data.drop(['Prod'], axis=1)

scaler = StandardScaler()
# подгоняем параметры стандартизатора (вычисляем среднее и СКО)
X = scaler.fit_transform(X)

# добавляем полиномиальные признаки
poly = PolynomialFeatures(degree=3, include_bias=False)
X = poly.fit_transform(X)

# создаём модель линейной регрессии c L1-регуляризацией
lasso = Lasso(alpha=5, max_iter=10000)
lasso.fit(X, y)
print(lasso.coef_[:10], '...')

#Оцените среднее значение метрики MAPE, используя кросс-валидацию на пяти фолдах.

# создаём модель линейной регрессии c L1-регуляризацией
lasso = Lasso(alpha=5, max_iter=10000)
# оцениваем качество модели на кросс-валидации
cv_results = cross_validate(lasso, X, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results['test_score'].mean() * 100))

[927.20383187 -88.21972516 375.12303358 121.02250022  70.46423802
   0.           8.12236776   0.          -3.04261735  -1.56030064] ...
MAPE на тренировочных фолдах: 1.83 %
MAPE на валидационных фолдах: 2.28 %


## Задание 3.3

In [64]:
#Обучите модель Ridge из библиотеки sklearn на полученных полиномиальных факторах, предварительно стандартизировав факторы с помощью StandardScaler. Коэффициент регуляризации выставите равным 1.
from sklearn.linear_model import Ridge

# создаём модель линейной регрессии c L2-регуляризацией
ridge = Ridge(alpha=1, solver='svd')
ridge.fit(X, y)
print(ridge.coef_[:10], '...')

#Оцените среднее значение метрики MAPE, используя кросс-валидацию на пяти фолдах.

# создаём модель линейной регрессии c L2-регуляризацией
ridge = Ridge(alpha=1, solver='svd')

# оцениваем качество модели на кросс-валидации
cv_results = cross_validate(ridge, X, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results['test_score'].mean() * 100))

[ 896.69341149 -132.70496963  369.483924    154.22971824   94.91001658
   25.53898554   18.32860477  -20.40707726    4.52167629   -4.0084608 ] ...
MAPE на тренировочных фолдах: 1.83 %
MAPE на валидационных фолдах: 2.72 %
